# Import Required Packages

In [ ]:
%%capture
!pip install -q -U keras-tuner

In [5]:
# basic packages
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#sklearn packages
from sklearn.metrics import (confusion_matrix, 
                             classification_report)

# tensorflow packages
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Dense, 
                                     LSTM, 
                                     BatchNormalization,
                                     Input)
from tensorflow.keras.optimizers import (Adam, 
                                         AdamW)
from tensorflow.keras.losses import (SparseCategoricalCrossentropy,
                                     CategoricalCrossentropy)

import keras_tuner as kt

# Load Train, Validation and Testing Numpy Arrays

In [9]:
data = np.load("../data/processed/processed_array.npz")

X_train, y_train = data["x_train"], data["y_train"]
X_val, y_val = data["x_val"], data["y_val"]
X_test, y_test = data["x_test"], data["y_test"]

print(f"Shape of X_train : {X_train.shape} and y_train : {y_train.shape}")
print(f"Shape of X_val : {X_val.shape} and y_train : {y_val.shape}")
print(f"Shape of X_test : {X_test.shape} and y_train : {y_test.shape}")

Shape of X_train : (5881, 128, 9) and y_train : (5881, 6)
Shape of X_val : (1471, 128, 9) and y_train : (1471, 6)
Shape of X_test : (2947, 128, 9) and y_train : (2947, 6)


# Tensorflow Dataset

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(64)

2025-07-01 20:21:48.244280: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-07-01 20:21:48.244498: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-07-01 20:21:48.244538: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
I0000 00:00:1751419308.245050  736250 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1751419308.245562  736250 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Keras Tuner Neural Network Architecture

In [ ]:
class tuner_model(Model):

    def __init__()